# 3 - Vertex AI > Pipelines - AutoML with clients (code) In automated pipeline

Use[ Kubeflow](https://www.kubeflow.org/) Pipelines running on [Vertex AI Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines/introduction) to orchestrate the process of training a custom model with AutoML Tabular and deploy it to a Vertex AI Endpoint for serving (online and batch) predictions and explanations.  This demonstrates how to automate the processes of (02a) or (02b) with pipeline orchestration.



### Prerequisites:
-  01 -  BigQuery - Table Data Source

### Overview:
-  Use Kubeflow Python SDK to build a pipeline
   -  Create pipeline using Google Cloud Pipeline Components (from google_cloud_pipeline_components import aiplatform as gcc_aip)
      -  Use cc_aip.TabularDatasetCreateOp to register dataset from BigQuery table
      -  Train AutoML tabular model with gcc_aip.AutoMLTabularTrainingJobRunOp
      -  Deploy model to endpoint using gcc_aip.ModelDeployOp
   -  Compile the pipeline
      -  kfp.v2.compiler.Compiler().compile
   -  Move the pipeline code to GCS Bucket
   -  Run the pipeline with google.cloud.aiplatform.PipelineJob
-  Online Predictions using Vertex AI Endpoint
-  Online Explanations using Vertex AI Endpoint
-  Batch Prediction Job for predictions and explanation with source and destination tables in BigQuery

### Resources:
-  [Vertex AI Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines/build-pipeline#google-cloud-components) see aiplatform.PipelineJob
-  [Python Client for Vertex AI](https://googleapis.dev/python/aiplatform/latest/aiplatform.html)
-  [Kubeflow Pipelines Components for Google Cloud](https://github.com/kubeflow/pipelines/tree/master/components/google-cloud)



---
## Setup

inputs:

In [1]:
REGION = 'us-central1'
PROJECT_ID='qwiklabs-gcp-04-24efb7132888'
DATANAME = 'fraud'
NOTEBOOK = '3'

# Resources
DEPLOY_COMPUTE = 'n1-standard-4'

# Model Training
VAR_TARGET = 'Class'
VAR_OMIT = 'transaction_id' # add more variables to the string with space delimiters

packages:

In [2]:
from google.cloud import aiplatform
from datetime import datetime
import kfp
#import kfp.v2.dsl as dsl
from google_cloud_pipeline_components import aiplatform as gcc_aip

from google.cloud import bigquery
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
import json
import numpy as np

clients:

In [3]:
aiplatform.init(project=PROJECT_ID, location=REGION)
bigquery = bigquery.Client()

parameters:

In [20]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET = "conchitademomlopsendtoend"
URI = f"gs://{BUCKET}/{DATANAME}/models/{NOTEBOOK}"
DIR = f"temp/{NOTEBOOK}"

In [21]:
# Give service account roles/storage.objectAdmin permissions
# Console > IMA > Select Account <projectnumber>-compute@developer.gserviceaccount.com > edit - give role
SERVICE_ACCOUNT = !gcloud config list --format='value(core.account)' 
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]
SERVICE_ACCOUNT

'948421954571-compute@developer.gserviceaccount.com'

environment:

In [22]:
!rm -rf {DIR}
!mkdir -p {DIR}

---
## Pipeline (KFP) Definition
- Flow
    - Create Vertex AI Dataset from link to BigQuery table
    - Create Vertex AI AutoML Tabular Training Job
    - Create Endpoint and Depoy trained model
    
Use [AI Platform Pipeline Components](https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-0.2.0/)
- Specifically, [AutoMLTabularTrainingJobRunOp](https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-0.2.0/google_cloud_pipeline_components.aiplatform.html#google_cloud_pipeline_components.aiplatform.AutoMLTabularTrainingJobRunOp)

Define a Job:
- Consider Weighting
- Model Type
- Optimization Objective

In [30]:
@kfp.dsl.pipeline(name = f'kfp-{NOTEBOOK}-{DATANAME}-{TIMESTAMP}', pipeline_root = URI+'/'+str(TIMESTAMP)+'/kfp/')
def pipeline(
    project: str = PROJECT_ID,
    dataname: str = DATANAME,
    display_name: str = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    deploy_machine: str = DEPLOY_COMPUTE,
    bq_source: str = f'bq://{PROJECT_ID}.{DATANAME}.{DATANAME}_prepped',
    var_target: str = VAR_TARGET,
    var_omit: str = VAR_OMIT,
    label: str = NOTEBOOK 
):
    
    # dataset
    dataset = gcc_aip.TabularDatasetCreateOp(
        project = project,
        display_name = display_name,
        bq_source = bq_source,
        #labels = {'notebook':f'{label}'}
    )
    
    # get feature names
    from google.cloud import bigquery
    bigquery = bigquery.Client(project = PROJECT_ID)
    query = f"SELECT * FROM {DATANAME}.INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{DATANAME}_prepped'"
    schema = bigquery.query(query).to_dataframe()
    OMIT = VAR_OMIT.split() + [VAR_TARGET, 'splits']
    features = schema[~schema.column_name.isin(OMIT)].column_name.tolist()
    features = dict.fromkeys(features, 'auto')
    
    # training
    model = gcc_aip.AutoMLTabularTrainingJobRunOp(
        project = project,
        display_name = display_name,
        optimization_prediction_type = "classification",
        optimization_objective = "maximize-au-prc",
        budget_milli_node_hours = 1000,
        disable_early_stopping=False,
        column_specs = features,
        dataset = dataset.outputs['dataset'],
        target_column = var_target,
        predefined_split_column_name = 'splits',
        #labels = {'notebook':f'{label}'}
    )
    
    # Endpoint: Creation
    endpoint = gcc_aip.EndpointCreateOp(
        project = project,
        display_name = display_name,
        #labels = {'notebook':f'{label}'}
    )
    
    # Endpoint: Deployment of Model
    deployment = gcc_aip.ModelDeployOp(
        model = model.outputs["model"],
        endpoint = endpoint.outputs["endpoint"],
        dedicated_resources_min_replica_count = 1,
        dedicated_resources_max_replica_count = 1,
        traffic_split = {"0": 100},
        dedicated_resources_machine_type= deploy_machine
    )

---
## Compile Pipeline

In [31]:
kfp.v2.compiler.Compiler().compile(
    pipeline_func = pipeline,
    package_path = f"{DIR}/{NOTEBOOK}.json"
)

Move compiled pipeline files to GCS Bucket

In [32]:
!gsutil cp {DIR}/{NOTEBOOK}.json {URI}/{TIMESTAMP}/kfp/

Copying file://temp/3/3.json [Content-Type=application/json]...
/ [1 files][ 23.4 KiB/ 23.4 KiB]                                                
Operation completed over 1 objects/23.4 KiB.                                     


---
## Create Vertex AI Pipeline Job

In [33]:
pipeline = aiplatform.PipelineJob(
    display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    template_path = f"{URI}/{TIMESTAMP}/kfp/{NOTEBOOK}.json",
    pipeline_root = f"{URI}/{TIMESTAMP}/kfp/",
    labels = {'notebook':f'{NOTEBOOK}'}
)

In [34]:
response = pipeline.run(service_account = SERVICE_ACCOUNT)

INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/948421954571/locations/us-central1/pipelineJobs/kfp-3-fraud-20220320113933-20220320115351
INFO:google.cloud.aiplatform.pipeline_jobs:To use this PipelineJob in another session:
INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/948421954571/locations/us-central1/pipelineJobs/kfp-3-fraud-20220320113933-20220320115351')
INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/kfp-3-fraud-20220320113933-20220320115351?project=948421954571
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/948421954571/locations/us-central1/pipelineJobs/kfp-3-fraud-20220320113933-20220320115351 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/948421954

In [35]:
aiplatform.get_pipeline_df(pipeline = f'kfp-{NOTEBOOK}-{DATANAME}-{TIMESTAMP}')

,pipeline_name,run_name,param.input:var_target,param.input:deploy_machine,param.input:display_name,param.input:label,param.input:bq_source,param.input:var_omit,param.input:dataname,param.input:project
0,kfp-3-fraud-20220320113933,kfp-3-fraud-20220320113933-20220320115351,Class,n1-standard-4,3_fraud_20220320113933,3,bq://qwiklabs-gcp-04-24efb7132888.fraud.fraud_...,transaction_id,fraud,qwiklabs-gcp-04-24efb7132888
1,kfp-3-fraud-20220320113933,kfp-3-fraud-20220320113933-20220320115030,Class,n1-standard-4,3_fraud_20220320113933,3,bq://qwiklabs-gcp-04-24efb7132888.fraud.fraud_...,transaction_id,fraud,qwiklabs-gcp-04-24efb7132888
2,kfp-3-fraud-20220320113933,kfp-3-fraud-20220320113933-20220320113943,Class,n1-standard-4,3_fraud_20220320113933,3,bq://qwiklabs-gcp-04-24efb7132888.fraud.fraud_...,transaction_id,fraud,qwiklabs-gcp-04-24efb7132888


---
## Evaluation
While the model above was trained using AutoML with the API, it is still possible to review the evaluation metrics directly in the Google Cloud Console.  Just visit the Models section of Vertex AI service and select the model and it will present the evaluation metrics with many helpful visuals.

It is also possible to retrieve the evaluation metrics for you model using the API.  This section shows how to use the API.


Get the Model:

In [36]:
model = aiplatform.Model.list(filter=f'display_name={NOTEBOOK}_{DATANAME}_{TIMESTAMP}')

Setup a model client for the model create by this notebook:

In [37]:
model[0].resource_name

'projects/948421954571/locations/us-central1/models/4135733823042224128'

In [38]:
model_client = aiplatform.gapic.ModelServiceClient(
    client_options = {
        'api_endpoint' : f'{REGION}-aiplatform.googleapis.com'
    }
)

Retrives the aggregate model evalution metrics for the model as a whole.  First, use `.list_model_evaluations` to retrieve the evaluation id, then use `.get_model_evaluation` for the evaluation id:

In [39]:
evaluations = model_client.list_model_evaluations(parent = model.resource_name)
evals = iter(evaluations)
eval_id = next(evals).name
geteval = model_client.get_model_evaluation(name = eval_id)

AttributeError: 'list' object has no attribute 'resource_name'

Review several of the metrics include in the evaluation.  Also, compare these to the results in the console view.

In [ ]:
geteval.metrics['auPrc']

In [ ]:
for i in range(len(geteval.metrics['confusionMatrix']['annotationSpecs'])):
    print('True Label = ', geteval.metrics['confusionMatrix']['annotationSpecs'][i]['displayName'], ' has Predicted labels = ', geteval.metrics['confusionMatrix']['rows'][i])

For models with labels you can retrieve the evaluation metrics for each slice of the model:

In [ ]:
slices = model_client.list_model_evaluation_slices(parent = eval_id)

In [ ]:
for slice in slices:
    print('Label = ', slice.slice_.value, 'has auPrc = ', slice.metrics['auPrc'])

---
## Prediction

### Prepare a record for prediction: instance and parameters lists

In [ ]:
pred = bigquery.query(query = f"SELECT * FROM {DATANAME}.{DATANAME}_prepped WHERE splits='TEST' LIMIT 10").to_dataframe()

In [ ]:
pred.head(4)

In [ ]:
newob = pred[pred.columns[~pred.columns.isin(VAR_OMIT.split()+[VAR_TARGET, 'splits'])]].to_dict(orient='records')[0]
#newob

Need to understand the format of variables that the predictions expect.  AutoML may convert the type of some variables. The following cells retrieve the model from the endpoint and its schemata:

In [ ]:
newob['Time'] = str(newob['Time'])

In [ ]:
instances = [json_format.ParseDict(newob, Value())]
parameters = json_format.ParseDict({}, Value())

### Get Predictions: Python Client

In [ ]:
aiplatform.Endpoint.list(filter=f'display_name={NOTEBOOK}_{DATANAME}_{TIMESTAMP}')

In [ ]:
endpoint = aiplatform.Endpoint.list(filter=f'display_name={NOTEBOOK}_{DATANAME}_{TIMESTAMP}')[0]
endpoint.display_name

In [ ]:
prediction = endpoint.predict(instances=instances, parameters=parameters)
prediction

In [ ]:
prediction.predictions[0]['classes'][np.argmax(prediction.predictions[0]['scores'])]

### Get Predictions: REST

In [ ]:
with open(f'{DIR}/request.json','w') as file:
    file.write(json.dumps({"instances": [newob]}))

In [ ]:
!curl -X POST \
-H "Authorization: Bearer "$(gcloud auth application-default print-access-token) \
-H "Content-Type: application/json; charset=utf-8" \
-d @{DIR}/request.json \
https://{REGION}-aiplatform.googleapis.com/v1/{endpoint.resource_name}:predict

### Get Predictions: gcloud (CLI)

In [ ]:
!gcloud beta ai endpoints predict {endpoint.name.rsplit('/',1)[-1]} --region={REGION} --json-request={DIR}/request.json

---
## Explanations
Interpretation Guide
- https://cloud.google.com/vertex-ai/docs/predictions/interpreting-results-automl#tabular

In [ ]:
explanation = endpoint.explain(instances=instances, parameters=parameters)

In [ ]:
explanation.predictions

In [ ]:
print("attribution:")
print("baseline output",explanation.explanations[0].attributions[0].baseline_output_value)
print("instance output",explanation.explanations[0].attributions[0].instance_output_value)
print("output_index",explanation.explanations[0].attributions[0].output_index)
print("output display value",explanation.explanations[0].attributions[0].output_display_name)
print("approximation error",explanation.explanations[0].attributions[0].approximation_error)

In [ ]:
import matplotlib.pyplot as plt
features = []
scores = []
for k in explanation.explanations[0].attributions[0].feature_attributions:
    features.append(k)
    scores.append(explanation.explanations[0].attributions[0].feature_attributions[k])
features = [x for _, x in sorted(zip(scores, features))]
scores = sorted(scores)
fig, ax = plt.subplots()
fig.set_size_inches(9, 9)
ax.barh(features, scores)
fig.show()

---
## Batch Predictions: BigQuery Source to BigQuery Destination, with Explanations

In [ ]:
batch = aiplatform.BatchPredictionJob.create(
    job_display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    model_name = endpoint.list_models()[0].model,
    instances_format = "bigquery",
    predictions_format = "bigquery",
    bigquery_source = f'bq://{PROJECT_ID}.{DATANAME}.{DATANAME}_prepped',
    bigquery_destination_prefix = f"{PROJECT_ID}",
    generate_explanation=True,
    labels = {'notebook':f'{NOTEBOOK}'}
)